In [2]:
import requests
import numpy as np
import json
import librosa
from scipy import stats
import tensorflow as tf
import base64


def compute_mfcc_features(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
    mean_mfcc = np.mean(mfcc)
    max_mfcc = np.max(mfcc)
    median_mfcc = np.median(mfcc)
    q1_mfcc = np.percentile(mfcc, 25)
    iqr_mfcc = stats.iqr(mfcc.flatten())
    minmax_mfcc = np.ptp(mfcc)
    kurtosis_mfcc = stats.kurtosis(mfcc.flatten())
    
    return {
        'Mean mfcc': mean_mfcc,
        'Max mfcc': max_mfcc,
        'Median mfcc': median_mfcc,
        'Quartile 1 mfcc': q1_mfcc,
        'IQR mfcc': iqr_mfcc,
        'MinMax mfcc': minmax_mfcc,
        'Kurtosis mfcc': kurtosis_mfcc
    }

def compute_chroma_stft_features(y, sr):
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    
    mean_chroma_stft = np.mean(chroma_stft)
    max_chroma_stft = np.max(chroma_stft)
    median_chroma_stft = np.median(chroma_stft)
    q1_chroma_stft = np.percentile(chroma_stft, 25)
    q3_chroma_stft = np.percentile(chroma_stft, 75)
    iqr_chroma_stft = stats.iqr(chroma_stft.flatten())
    minmax_chroma_stft = np.ptp(chroma_stft)
    kurtosis_chroma_stft = stats.kurtosis(chroma_stft.flatten())
    
    return {
        'Mean chroma_stft': mean_chroma_stft,
        'Max chroma_stft': max_chroma_stft,
        'Median chroma_stft': median_chroma_stft,
        'Quartile 1 chroma_stft': q1_chroma_stft,
        'Quartile 3 chroma_stft': q3_chroma_stft,
        'IQR chroma_stft': iqr_chroma_stft,
        'MinMax chroma_stft': minmax_chroma_stft,
        'Kurtosis chroma_stft': kurtosis_chroma_stft
    }

def compute_chroma_cqt_features(y, sr):
    chroma_cqt = librosa.feature.chroma_cqt(y=y, sr=sr)
    
    mean_chroma_cqt = np.mean(chroma_cqt)
    max_chroma_cqt = np.max(chroma_cqt)
    median_chroma_cqt = np.median(chroma_cqt)
    q1_chroma_cqt = np.percentile(chroma_cqt, 25)
    q3_chroma_cqt = np.percentile(chroma_cqt, 75)
    iqr_chroma_cqt = stats.iqr(chroma_cqt.flatten())
    minmax_chroma_cqt = np.ptp(chroma_cqt)
    kurtosis_chroma_cqt = stats.kurtosis(chroma_cqt.flatten())
    
    return {
        'Mean chroma_cqt': mean_chroma_cqt,
        'Max chroma_cqt': max_chroma_cqt,
        'Median chroma_cqt': median_chroma_cqt,
        'Quartile 1 chroma_cqt': q1_chroma_cqt,
        'Quartile 3 chroma_cqt': q3_chroma_cqt,
        'IQR chroma_cqt': iqr_chroma_cqt,
        'MinMax chroma_cqt': minmax_chroma_cqt,
        'Kurtosis chroma_cqt': kurtosis_chroma_cqt
    }

def compute_rms_features(y):
    rms = librosa.feature.rms(y=y)
    
    mean_rms = np.mean(rms)
    max_rms = np.max(rms)
    median_rms = np.median(rms)
    q1_rms = np.percentile(rms, 25)
    iqr_rms = stats.iqr(rms.flatten())
    minmax_rms = np.ptp(rms)
    kurtosis_rms = stats.kurtosis(rms.flatten())
    
    return {
        'Mean rms': mean_rms,
        'Max rms': max_rms,
        'Median rms': median_rms,
        'Quartile 1 rms': q1_rms,
        'IQR rms': iqr_rms,
        'MinMax rms': minmax_rms,
        'Kurtosis rms': kurtosis_rms
    }

def convert_all_values_to_float(input_dict):
    return {k: float(v) for k, v in input_dict.items()}


y, sr = librosa.load('_201012172012_.wav')


mfcc_features = compute_mfcc_features(y, sr)
chroma_stft_features = compute_chroma_stft_features(y, sr)
chroma_cqt_features = compute_chroma_cqt_features(y, sr)
rms_features = compute_rms_features(y)

input = {}

input.update(mfcc_features)
input.update(chroma_stft_features)
input.update(chroma_cqt_features)
input.update(rms_features)

input = convert_all_values_to_float(input)

def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "heartbeat_training": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(input)

endpoint = "https://agile-vibrancy-production.up.railway.app/v1/models/heartbeat-classification-model:predict"
response = requests.post(endpoint, data=json_data)

labels = ['normal', 'murmur', 'artifact', 'extrastole', 'extrahls']

prediction = response.json().get("predictions")

pred = prediction[0]

max_value = max(pred)
max_index = pred.index(max_value)

predicted_label = labels[max_index]

# Tampilkan hasil
print(f"Label prediksi: {predicted_label}")


Label prediksi: artifact
